In [1]:
#imports
import pandas as pd
import ftplib
import sys
import os
import shutil
import pyPortal as p
import time
import glob
from bi_tools import bi

#check box connection
try:
    import box_config as b
    ftp = ftplib.FTP("ftp.box.com")
    ftp.login(b.box['user'],b.box['password'])
    ftp.quit()
except:
    print 'Box credentials needed'
    for attempt in range(10):
        try:
            username = raw_input('username (with email): ')
            password = raw_input('password: ')
            fname = sys.path[2]+'/box_config.py'
            box_info = {'user':username,
                        'password':password}
            config_data = 'box = '+str(box_info)
            with open(fname, 'w') as f:
                f.write(config_data)
            import box_config as b
            reload(b)
            ftp = ftplib.FTP("ftp.box.com")
            ftp.login(b.box['user'],b.box['password'])
            ftp.quit()
            print 'Cedentials for {} validated and saved.'.format(b.box['user'])
            break
        except ftplib.error_perm:
            if attempt == 9:
                print 'Too many password attempts, consider contacting IT'
                os.remove(fname)
                raw_input('Press enter to exit')
                exit()
            print 'Credentials invalid, please re-try.'
            continue

ImportError: No module named pyPortal

In [30]:
# read in data
df = pd.read_excel('srp2_accounts.xlsx')
df.columns = ['account']
df.head()

,account
0,2967292
1,3000691
2,2936008
3,3009183
4,3014746


In [31]:
# Looks like there are some dupes
print df.shape
print len(df['account'].unique())
df = df.drop_duplicates()

(158, 1)
95


In [32]:
#fetch additional details
query = '''
select
p.ACCOUNT_NUMBER__C as 'account'
, p.ID
, p.NAME
, p.SITE_STATE_PROVINCE__C
, p.STATUS__C
from
salesforce_prod.milestone1_project__c as p
where p.ACCOUNT_NUMBER__C in {}
'''.format(tuple(df['account']))

df2 = bi.query_bi(query=query)
df2['account'] = df2['account'].astype(int)

df = df.merge(df2, how='left', on='account')

Connected to Bi
Query completed in 0.2040 seconds
95 rows returned
Closed connection to Bi


In [33]:
#limit to NJ and CT
df = df[df['SITE_STATE_PROVINCE__C'].isin(['NJ','CT'])]
df.reset_index(drop=True, inplace=True)

In [34]:
def pto(project, dfolder):
    #old PTO function
    if not os.path.exists(dfolder):
        os.makedirs(dfolder)

    ftp = ftplib.FTP("ftp.box.com")
    ftp.login(b.box['user'],b.box['password'])

    if isinstance(project, list):

        for proj in project:
            customer = proj[:1]
            folder = '/Salesforce/{} Customers/{}/2 Incentive & Interconnection/Interconnection/Final'.format(customer, proj)
            fname = '{}_SystemCommissioned.pdf'.format(proj)
            try:
                ftp.cwd(folder)
            except:
                print 'Could not find: {}'.format(folder)

            try:
                fhandle = open('{}/{}'.format(dfolder,fname),'wb')
                ftp.retrbinary('RETR ' + fname, fhandle.write)
                fhandle.close()
            except:
                print 'Could not download: {}'.format(fname)

    elif isinstance(project, basestring):
        customer = project[:1]
        folder = '/Salesforce/{} Customers/{}/2 Incentive & Interconnection/Interconnection/Final'.format(customer, project)
        fname = '{}_SystemCommissioned.pdf'.format(project)
        try:
            ftp.cwd(folder)
        except:
            print 'Could not find: {}'.format(folder)

        try:
            fhandle = open('{}/{}'.format(dfolder,fname),'wb')
            ftp.retrbinary('RETR ' + fname, fhandle.write)
            fhandle.close()
        except:
            print 'Could not download: {}'.format(fname)

    else:
        print 'Invalid project given'

    ftp.quit()
    
def get_box_file(ftp_con, target_path, download_dir):
    
    # setup params
    result = None
    target_file = os.path.basename(target_path)
    target_dir = target_path[:-len(target_file)]
    
    if not os.path.exists(download_dir + target_file):
        
    
        # reset to home_dir
        ftp_con.cwd('/')

        try:
            # move to target directory
            ftp_con.cwd(target_dir)

        except:

            # report result if failure to find dir
            result = 'Could not find directory.'

        if result == None:

            # if no faults then attempt download
            fhandle = open(download_dir + target_file, 'wb')

            try:
                ftp_con.retrbinary('RETR ' + target_file, fhandle.write)
                fhandle.close()
                result = 'Success.'

            except:
                # if failure then remove file that was created
                fhandle.close()
                result = 'No file found.'
                os.remove(download_dir + target_file)
    else:
        result = 'File already downloaded'
    
    return result

def get_box_dir(ftp_con, target_dir, download_dir):
    
    # setup params
    result = None
    
    if not os.path.exists(download_dir + os.path.basename(target_dir)):
        
    
        # reset to home_dir
        ftp_con.cwd('/')

        try:
            # move to target directory
            ftp_con.cwd(target_dir)

        except:

            # report result if failure to find dir
            result = 'Could not find directory.'

        if result == None:

            # if no faults then attempt download
            file_names = ftp.nlst()
            
            if len(file_names) > 0:
                os.makedirs(download_dir + os.path.basename(target_dir))
                files = 0
                for fname in file_names:
                    if fname[-3:] == 'pdf':
                        d_path = download_dir + os.path.basename(target_dir) + '/' + fname
                        
                        fhandle = open(d_path, 'wb')
                        ftp_con.retrbinary('RETR ' + fname, fhandle.write)
                        fhandle.close()
                        files += 1

                result = 'Success {} files in dir'.format(files)

            
    else:
        result = 'File already downloaded'
    
    return result

In [35]:
working_dir = 'priority2/'

if not os.path.exists(working_dir):
    os.makedirs(working_dir)
    print 'made new doc directory'

made new doc directory


## Grab files from box

In [36]:
ftp = ftplib.FTP("ftp.box.com")
ftp.login(b.box['user'],b.box['password'])

for index, row in df.iterrows():
    skip = False
    # set up base params
    start = time.time()
    # directories
    project = row['NAME']
    
    try:
        print "Downloading files for: {}".format(project)
    
    except:
        print "Could not get files for {} ascii character".format(row['account'])
        skip = True
    
    if not skip:
        try:
            box_dir = '/Salesforce/{} Customers/{}/'.format(project[:1], project)

        except:
            continue

        download_dir = working_dir + project + '/'

        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        if row['SITE_STATE_PROVINCE__C'] == 'CT':

            #Audit
            f_path = '2 Incentive & Interconnection/Incentive/{}_Audit.pdf'.format(project)
            target_path = box_dir + f_path
            df.loc[index, 'Audit'] = get_box_file(ftp, target_path, download_dir)

            #PTO
            f_path = '2 Incentive & Interconnection/Interconnection/Final/{}_SystemCommissioned.pdf'.format(project)
            target_path = box_dir + f_path
            df.loc[index, 'PTO'] = get_box_file(ftp, target_path, download_dir)

            #BOM
            f_path = '{} - Home Visit & Installation/{}_BOM.pdf'.format(project, project)
            target_path = box_dir + f_path
            df.loc[index, 'BOM'] = get_box_file(ftp, target_path, download_dir)

            #SAR
            dir_path = '{} - Design/Solar Access Reports'.format(project)
            target_dir = box_dir + dir_path
            df.loc[index, 'SAR'] = get_box_dir(ftp, target_dir, download_dir)

            #UBx
            f_path = '2 Incentive & Interconnection/Interconnection/{}_UBx1.pdf'.format(project)
            target_path = box_dir + f_path
            df.loc[index, 'UBx1'] = get_box_file(ftp, target_path, download_dir)

            #UBx1stiched
            f_path = '2 Incentive & Interconnection/Interconnection/{}_UBx1stitched.pdf'.format(project)
            target_path = box_dir + f_path
            df.loc[index, 'UBx1stitched '] = get_box_file(ftp, target_path, download_dir)

            #UBx1stiched
            f_path = '{} - Home Visit & Installation/{}_PS_X1.pdf'.format(project, project)
            target_path = box_dir + f_path
            df.loc[index, 'PS_X1'] = get_box_file(ftp, target_path, download_dir)

        elif row['SITE_STATE_PROVINCE__C'] == 'NJ':

            #psx1
            f_path = '{} - Home Visit & Installation/{}_PS_X1.pdf'.format(project, project)
            target_path = box_dir + f_path
            df.loc[index, 'PS_X1'] = get_box_file(ftp, target_path, download_dir)

            #PTO
            f_path = '2 Incentive & Interconnection/Interconnection/Final/{}_SystemCommissioned.pdf'.format(project)
            target_path = box_dir + f_path
            df.loc[index, 'PTO'] = get_box_file(ftp, target_path, download_dir)

            #RRF
            f_path = '2 Incentive & Interconnection/Incentive/{}_RRF.pdf'.format(project)
            target_path = box_dir + f_path
            df.loc[index, 'RRF'] = get_box_file(ftp, target_path, download_dir)

            #installagree
            f_path = '2 Incentive & Interconnection/Incentive/{}_InstallAgree.pdf'.format(project)
            target_path = box_dir + f_path
            df.loc[index, 'InstallA'] = get_box_file(ftp, target_path, download_dir)

            #installagree
            f_path = '2 Incentive & Interconnection/Incentive/{}_InstallAgmt.pdf'.format(project)
            target_path = box_dir + f_path
            df.loc[index, 'InstallAgmt'] = get_box_file(ftp, target_path, download_dir)

        end = time.time()
        print "project files downlaoded in {} seconds".format(end-start)
        print "{}/{} project files downloaded".format(index+1,df.shape[0])

ftp.close()

project files downlaoded in 10.5409998894 seconds
1/79 project files downloaded
project files downlaoded in 7.50200009346 seconds
2/79 project files downloaded
project files downlaoded in 7.66100001335 seconds
3/79 project files downloaded
project files downlaoded in 3.76099991798 seconds
4/79 project files downloaded
project files downlaoded in 9.98300004005 seconds
5/79 project files downloaded
project files downlaoded in 13.5409998894 seconds
6/79 project files downloaded
project files downlaoded in 2.96600008011 seconds
7/79 project files downloaded
project files downlaoded in 11.9190001488 seconds
8/79 project files downloaded
project files downlaoded in 3.97799992561 seconds
9/79 project files downloaded
project files downlaoded in 4.91100001335 seconds
10/79 project files downloaded
project files downlaoded in 8.2990000248 seconds
11/79 project files downloaded
project files downlaoded in 8.58799982071 seconds
12/79 project files downloaded
project files downlaoded in 3.48900008

In [22]:
df.head()

,account,ID,NAME,SITE_STATE_PROVINCE__C,STATUS__C,PS_X1,PTO,RRF,InstallA,InstallAgmt,Audit,BOM,SAR,UBx1,UBx1stitched
0,2967292,a0w0P00000HReddQAD,Gonzalez - 2967292,NJ,Active,Success.,No file found.,Success.,No file found.,Success.,NaN,NaN,NaN,NaN,NaN
1,3000691,a0w0P00000HRgEiQAL,Alvi - 3000691,NJ,Active,Success.,No file found.,Success.,No file found.,Success.,NaN,NaN,NaN,NaN,NaN
2,2936008,a0w0P00000HRh6AQAT,Farrell - 2936008,NJ,Active,Success.,No file found.,Success.,No file found.,Success.,NaN,NaN,NaN,NaN,NaN
3,3009183,a0w0P00000HRhuEQAT,Granson - 3009183,NJ,Active,Success.,No file found.,Success.,No file found.,Success.,NaN,NaN,NaN,NaN,NaN
4,3014746,a0w0P00000HRkFUQA1,Hirday - 3014746,NJ,Active,Success.,No file found.,Success.,No file found.,Success.,NaN,NaN,NaN,NaN,NaN


# SF pull

In [37]:
def get_sf_file(portal, project_id, target_file, download_path):
    result = None
    portal.load_page('https://sungevity.my.salesforce.com/' + project_id)
    try:
        portal.click_link_text(target_file, raise_error=True)
    except:
        result = 'Could not find file.'
    
    if result == None:
        portal.click_link_text('View file')
        time.sleep(5)
        last_download = max(glob.iglob('C:\Users\elamarre\Desktop\SREC_docs\*.pdf'), key=os.path.getctime)
        os.rename(last_download, download_path + os.path.basename(last_download))
        result = 'SUCCESS'
    return result

In [38]:
portal = p.Portal('https://sungevity.my.salesforce.com/home/home.jsp', chrome=True)

In [39]:
working_folder = working_dir

#portal = p.Portal('https://sungevity.my.salesforce.com/home/home.jsp', chrome=True)

for index, row in df.iterrows():
    skip = False
    # set up base params
    start = time.time()
    # directories
    project_name = row['NAME']
    project_id = row['ID']
    
    try:
        print "Downloading files for: {}".format(project_name)
    
    except:
        print "Could not get files for {} ascii character".format(row['account'])
        skip = True
    
    if not skip:

        download_dir = working_folder + project_name + '/'
        
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        if row['SITE_STATE_PROVINCE__C'] == 'CT':
            
            #post PTO doc
            fname = project_name + '_post_PTO_doc.pdf'
            df.loc[index, 'post_pto'] = get_sf_file(portal, row['ID'], fname, download_dir)
            
            #icf_completed
            fname = project_name + '_ICF_Completed.pdf'
            df.loc[index, 'ICF_complete'] = get_sf_file(portal, row['ID'], fname, download_dir)
            
        if row['SITE_STATE_PROVINCE__C'] == 'NJ':
            
            #post PTO doc
            fname = project_name + '_post_PTO_doc.pdf'
            df.loc[index, 'post_pto'] = get_sf_file(portal, row['ID'], fname, download_dir)
            
    end = time.time()
    print "project files downlaoded in {} seconds".format(end-start)
    print "{}/{} project files downloaded".format(index+1,df.shape[0])

portal.close()

project files downlaoded in 4.28500008583 seconds
1/79 project files downloaded
project files downlaoded in 4.24399995804 seconds
2/79 project files downloaded
project files downlaoded in 2.92200016975 seconds
3/79 project files downloaded
project files downlaoded in 2.87999987602 seconds
4/79 project files downloaded
project files downlaoded in 3.10600018501 seconds
5/79 project files downloaded
project files downlaoded in 3.19599986076 seconds
6/79 project files downloaded
project files downlaoded in 2.91700005531 seconds
7/79 project files downloaded
project files downlaoded in 3.08299994469 seconds
8/79 project files downloaded
project files downlaoded in 2.94000005722 seconds
9/79 project files downloaded
project files downlaoded in 3.59299993515 seconds
10/79 project files downloaded
project files downlaoded in 2.9350001812 seconds
11/79 project files downloaded
project files downlaoded in 3.09399986267 seconds
12/79 project files downloaded
project files downlaoded in 2.94500017

# S drive

In [42]:
working_folder = working_dir

#portal = p.Portal('https://sungevity.my.salesforce.com/home/home.jsp', chrome=True)

for index, row in df.iterrows():
    skip = False
    # set up base params
    start = time.time()
    # directories
    project_name = row['NAME']
    project_id = row['ID']
    
    try:
        print "Downloading files for: {}".format(project_name)
    
    except:
        print "Could not get files for {} ascii character".format(row['account'])
        skip = True
    
    if not skip:

        download_dir = working_folder + project_name + '/'
        
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

            
        if row['SITE_STATE_PROVINCE__C'] == 'NJ':
            
            gen_target_path = 'S:/Rebate and Interconnection/Forms/New Jersey/NJCEP SPF/Signed by SR/'
            download_path = download_dir + project_name + '_signed_reg.pdf'

            try:
                shutil.copy(gen_target_path + project_name + '.pdf', download_path)
                df.loc[index, 'signed_reg'] = 'SUCCESS'
            except:
                df.loc[index, 'signed_reg'] = 'Could not find file.'
            
    end = time.time()
    print "project files downlaoded in {} seconds".format(end-start)
    print "{}/{} project files downloaded".format(index+1,df.shape[0])

project files downlaoded in 0.0 seconds
1/79 project files downloaded
project files downlaoded in 0.0 seconds
2/79 project files downloaded
project files downlaoded in 0.0 seconds
3/79 project files downloaded
project files downlaoded in 0.00300002098083 seconds
4/79 project files downloaded
project files downlaoded in 0.00200009346008 seconds
5/79 project files downloaded
project files downlaoded in 0.00300002098083 seconds
6/79 project files downloaded
project files downlaoded in 0.0019998550415 seconds
7/79 project files downloaded
project files downlaoded in 0.0019998550415 seconds
8/79 project files downloaded
project files downlaoded in 0.00200009346008 seconds
9/79 project files downloaded
project files downlaoded in 0.00200009346008 seconds
10/79 project files downloaded
project files downlaoded in 0.0019998550415 seconds
11/79 project files downloaded
project files downlaoded in 0.00200009346008 seconds
12/79 project files downloaded
project files downlaoded in 0.0020000934600

In [44]:
df.to_csv('sunrunp2_results.csv', encoding='utf')